# Model the Data
Given a set of scalar data, you can model it with a suitable pdf. This exercise will help you learn the following:
- How would you choose the best model to fit the given data?
- How would you estimate the model parameters from the given data?
- Given a model, how do you sample new data from it?

Note: You are allowed to use only numpy and matplotlib libraries. No ML library.

In [1]:
import numpy as np
np.random.seed(111)

In [2]:
def data2gaussian(S):
    '''
    Return optimal parameters - (mu,sigma)
    Inputs:
        S: np array of shape (Ns,). These are samples of a random variable.
    Outputs:
        mu: float
        sigma: float
    '''

    ### WRITE YOUR CODE HERE - 5 MARKS
    mu = np.mean(S)
    sigma = np.mean(np.square(S-mu))
    return mu, sigma

In [3]:
def test_data2gaussian(): # checks formatting only
    S = [0.1,-0.2,0.4,0,0]
    mu, sigma = data2gaussian(S)
    print(mu,sigma)
    assert isinstance(mu, (int, float))
    assert isinstance(sigma, (int, float))
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_data2gaussian()

0.06000000000000001 0.038400000000000004
Test passed 👍


In [4]:
def data2laplacian(S):
    '''
    Return optimal parameters - (mu,b). See https://en.wikipedia.org/wiki/Laplace_distribution
    Inputs:
        S: np array of shape (Ns,). These are samples of a random variable.
    Outputs:
        mu: float
        b: float
    '''

    ### WRITE YOUR CODE HERE - 5 MARKS
    mu = np.median(S)
    b = np.mean(np.absolute(S-mu))
    return mu, b

In [5]:
def test_data2laplacian(): # checks formatting only
    S = [0.1,-0.2,0.4,0,0]
    mu, b = data2laplacian(S)
    print(mu,b)
    assert isinstance(mu, (int, float))
    assert isinstance(b, (int, float))
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_data2laplacian()

0.0 0.14
Test passed 👍


In [6]:
def data2uniform(S):
    '''
    Return optimal parameters - (a,b)
    Inputs:
        S: np array of shape (Ns,). These are samples of a random variable.
    Outputs:
        a: float
        b: float
    '''

    ### WRITE YOUR CODE HERE - 5 MARKS
    b = np.max(S)
    a = np.min(S)
    return a, b 

In [7]:
def test_data2uniform(): # checks formatting only
    S = [0.1,-0.2,0.4,0,0]
    a, b = data2uniform(S)
    print(a,b)
    assert isinstance(a, (int, float))
    assert isinstance(b, (int, float))
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_data2uniform()

-0.2 0.4
Test passed 👍


In [13]:
# Distributions
def gaussian(x, mu, sigma):
    return np.exp(-0.5*((x-mu)/sigma)**2)/(sigma*np.sqrt(2*np.pi))

def laplacian(x, mu, b):
    return np.exp(-np.abs(x-mu)/b)/(2*b)

def uniform(x, a, b):
    return 1/(b-a)

# Likelihoods
def gaussian_likelihood(S, mu, sigma):
    p=1
    for n in range(len(S)):
        p*=gaussian(S[n], mu, sigma)
    return p

def laplacian_likelihood(S, mu, b):
    p=1
    for n in range(len(S)):
        p*=laplacian(S[n], mu, b)
    return p

def uniform_likelihood(S, a, b):
    p=1
    for n in range(len(S)):
        p*=uniform(S[n], a, b)
    return p

# Choose the model which maximises the likelihood
def data2model(S):
    '''
    Inputs:
        S: np array of shape (Ns,). These are scalar samples of a random variable.
    Outputs:
        modelName: return one out of these - 'gaussian', 'laplacian', or 
                   'uniform' which best models the data
    '''

    ### WRITE YOUR CODE HERE - 10 MARKS    
    # gaussian
    mu, sigma = data2gaussian(S)
    gauss_like = gaussian_likelihood(S, mu, sigma)
    # laplacian
    mu, b = data2laplacian(S)
    lap_like = laplacian_likelihood(S, mu, b)
    # uniform
    a, b = data2uniform(S)
    uni_like = uniform_likelihood(S, a, b)

    list_of_likelihoods = [gauss_like, lap_like, uni_like]
    list_of_modelNames = ['gaussian', 'laplacian', 'uniform']
    modelName = list_of_modelNames[np.argmax(list_of_likelihoods)]
    print(modelName)
    return modelName

In [14]:
def test_data2model(): # checks formatting only
    S = [0.1,-0.2,0.4,0,0]
    modelName = data2model(S)
    assert modelName in ['gaussian', 'laplacian', 'uniform']
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_data2model()

uniform
Test passed 👍


In [15]:
def sampleGMM(pi, mu, sigma, Ns=1):
    '''
    Inputs:
        pi: np.array of shape (K,), p(z_k)
        mu: np.array of shape (K,), mu of kth gaussian
        sigma: np.array of shape (K,), sigma of kth gaussian
        Ns: int, number of samples
    Outputs:
        S: np.array of shape (Ns,), samples from the GMM
    '''
    ### WRITE YOUR CODE HERE - 10 MARKS
    # Create array of pairs of mu's and sigma's of length Ns
    indices = []
    for k in range(len(pi)): indices.append(k)
    sampled_indices = np.random.choice(indices, size=Ns, p=pi)
    
    # Obtain a single sample from each of the Ns mu and sigma pairs
    S = []
    for n in range(Ns):
        S = np.concatenate((S, np.random.normal(loc=mu[sampled_indices[n]], scale=sigma[sampled_indices[n]], size=1)), axis=0)
    return S

In [16]:
def test_sampleGMM(): # checks formatting only
    pi = [0.3,0.7]
    mu = [-1.1, 1.3]
    sigma = [1.5, 0.4]
    Ns = 5
    S = sampleGMM(pi, mu, sigma, Ns)
    assert S.shape==(5,)
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_sampleGMM()

Test passed 👍


In [70]:
# 2dgaussian
def twodgaussian(n, k, S, mu, sigma):
    mu_k = mu[k]
    sigma_k = sigma[k]
    d = S.shape[1]
    return np.exp(-0.5*np.dot(S[n].T-mu_k.T, np.dot(np.linalg.inv(sigma_k), S[n]-mu_k)))/(((np.sqrt(2*np.pi))**d)*np.linalg.det(sigma_k))

# Y_nk
def create_gamma_nk(n, k, S, pi, mu, sigma):
    K = pi.shape[0]
    denominator=0
    for k_prime in range(K):
        if(k==k_prime) : numerator = pi[k_prime]*twodgaussian(n, k_prime, S, mu, sigma)
        denominator+=pi[k_prime]*twodgaussian(n, k_prime, S, mu, sigma)
    if(denominator==0) : return 0
    else : return numerator/denominator

# Y
def create_gamma(Ns, K, S, pi, mu, sigma):
    Y = np.zeros((Ns, K))
    for n in range(Ns):
        for k in range(K): Y[n][k]=create_gamma_nk(n, k, S, pi, mu, sigma)
    return Y

# pi
def update_pi(Ns, K, Y, pi):
    for k in range(K): 
        N_k = np.sum(Y[:, k])
        pi[k]=N_k/Ns

# mu
def update_mu(Ns, K, Y, S, mu):
    for k in range(K):
        N_k = np.sum(Y[:, k])
        for n in range(Ns):
            if(n==0) : numerator = Y[n][k]*S[n]
            else : numerator += Y[n][k]*S[n]
        mu[k]=numerator/N_k
    
# sigma
def update_sigma(Ns, K, Y, S, mu, sigma):
    for k in range(K):
        N_k = np.sum(Y[:, k])
        for n in range(Ns):
            if(n==0) : numerator = Y[n][k]*np.outer(S[n]-mu[k], S[n]-mu[k])
            else : numerator += Y[n][k]*np.outer(S[n]-mu[k], S[n]-mu[k])
        sigma[k]=numerator/N_k

# update
def update(max_iter, threshold, Ns, K, S, pi, mu, sigma):
    for iteration in range(max_iter):
        pi_orig=pi
        mu_orig=mu
        sigma_orig=sigma
        # Original
        Y = create_gamma(Ns, K, S, pi, mu, sigma)
        update_pi(Ns, K, Y, pi)
        update_mu(Ns, K, Y, S, mu)
        update_sigma(Ns, K, Y, S, mu, sigma)
        # Updated
        pi_upd=pi
        mu_upd=mu
        sigma_upd=sigma
        if(np.all(abs(pi_upd-pi_orig))<threshold or np.all(abs(mu_upd-mu_orig))<threshold 
           or np.all(abs(sigma_upd-sigma_orig))<threshold) : return
    
def data2GMM(S, K):
    '''
    Return optimal parameters - (pi,mu,sigma)
    Inputs:
        S: np array of shape (Ns,Na). These are samples of a random variable. Na can be 1, 2 or 3
    Outputs:
        pi: np array of shape (K,)
        mu: np array of shape (K, Na)
        sigma: np array of shape (K,Na,Na)
    '''
    ### WRITE YOUR CODE HERE - 15 MARKS
    Ns = S.shape[0]
    Na = S.shape[1]
    pi = np.random.uniform(low=0.0, high=2.0, size=(K))
    mu = np.random.uniform(low=0.0, high=2.0, size=(K, Na))
    sigma = np.random.uniform(low=0.0, high=2.0, size=(K, Na, Na))
    max_iter=10000
    threshold=1e-3
    update(max_iter, threshold, Ns, K, S, pi, mu, sigma)
    return pi, mu, sigma

In [71]:
def test_data2GMM(): # checks formatting only
    S = np.random.random((10,3))
    pi, mu, sigma = data2GMM(S,2)
    assert pi.shape==(2,)
    assert mu.shape==(2,3)
    assert sigma.shape==(2,3,3)
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_data2GMM()

Test passed 👍
